In [1]:
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Plots, IndexedGraphs, Statistics, InvertedIndices
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [24]:
T = 10
N = 100
k = 4
λ = 0.15
ρ = 0.25
γ = 0.1;

In [25]:
require_convergence = false
maxiter = 100;

In [26]:
matrix_sizes = [1, 2, 4, 6]
nobs = N÷3
infected_true = Vector{Int64}[]
infected_guess = Vector{Vector{Int64}}[]
Xs = Matrix{Int64}[];

In [27]:
ninstances = 20
for n in 1:ninstances
    println("Instance $n of $ninstances")
    gg = random_regular_graph(N, k)
    g = IndexedGraph(gg)
    bp = mpbp(SIS(g, λ, ρ, T; γ))
    X, _ = onesample(bp)
    push!(Xs, X)
    _, observed_times_nodes = draw_node_observations!(bp.ϕ, X, nobs, last_time=true)
    observed_nodes = [obs[1] for obs in observed_times_nodes]
    not_observed = (1:N)[Not(observed_nodes)]
    push!(infected_true, findall(isequal(INFECTED), X[:,end]))
    push!(infected_guess, Vector{Int64}[])
    for a in eachindex(matrix_sizes)
        d = matrix_sizes[a]
        svd_trunc = TruncBond(d)
        println("\tRun $a of $(length(matrix_sizes)). Matrix size $d")
        reset!(bp; observations=false)
        iters, _ = iterate!(bp; maxiter, tol=1e-3, svd_trunc, cb=CB_BP(bp; showprogress=false))
        if require_convergence
            iters == maxiter && error("BP did not converge")
        end
        b = beliefs(bp)
        push!(infected_guess[n], sort(not_observed; by=i->b[i][end][SUSCEPTIBLE]))
    end
end

Instance 1 of 20
	Run 1 of 4. Matrix size 1


	Run 2 of 4. Matrix size 2


In [ ]:
aucs_avg = [mean(auc(infected_guess[n][a], infected_true[n]) for n in 1:ninstances) 
    for a in eachindex(matrix_sizes)]
aucs_std = [std(auc(infected_guess[n][a], infected_true[n]) for n in 1:ninstances) 
    for a in eachindex(matrix_sizes)] ./ sqrt(length(infected_guess)); 

In [ ]:
plot(matrix_sizes, aucs_avg, ribbon=aucs_std, xlabel="matrix size", ylabel="AUC", label="", m=:o)